In [6]:
#!/usr/bin/env python
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2022-03-13 08:18:42
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0


import angr, monkeyhex


project = angr.Project('examples/fauxware/fauxware', auto_load_libs=False)

state = project.factory.entry_state()
sim_mgr = project.factory.simgr(state)
sim_mgr.active

[<SimState @ 0x400580>]

In [7]:
block = project.factory.block(project.entry)
block.pp()

        _start:
400580  xor     ebp, ebp
400582  mov     r9, rdx
400585  pop     rsi
400586  mov     rdx, rsp
400589  and     rsp, 0xfffffffffffffff0
40058d  push    rax
40058e  push    rsp
40058f  mov     r8, __libc_csu_fini
400596  mov     rcx, __libc_csu_init
40059d  mov     rdi, main
4005a4  call    __libc_start_main


In [8]:
sim_mgr.step()

WARNING | 2022-03-13 08:56:01,607 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff7c with 4 unconstrained bytes referenced from 0x400585 (_start+0x5 in fauxware (0x400585))


<SimulationManager with 1 active>

In [9]:
sim_mgr.active

[<SimState @ 0x400540>]

In [12]:
sim_mgr.active[0].block().pp()

        __libc_start_main:
400540  jmp     qword ptr [0x601018]


In [13]:
while len(sim_mgr.active) == 1:
    sim_mgr.step()

sim_mgr

WARNING | 2022-03-13 09:00:37,162 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff60 with 8 unconstrained bytes referenced from 0x700020 (strcmp+0x0 in extern-address space (0x20))


<SimulationManager with 2 active>

In [14]:
sim_mgr.active

[<SimState @ 0x400692>, <SimState @ 0x400699>]

In [15]:
sim_mgr.run()
sim_mgr

WARNING | 2022-03-13 09:02:55,510 | angr.state_plugins.posix | Trying to open unknown file b'\x00\x00\x00\x00\x00\x00\x00\x00' - created a symbolic file since ALL_FILES_EXIST is set
WARNING | 2022-03-13 09:02:55,568 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffefef0 with 8 unconstrained bytes referenced from 0x700010 (read+0x0 in extern-address space (0x10))
WARNING | 2022-03-13 09:02:55,613 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff20 with 8 unconstrained bytes referenced from 0x700020 (strcmp+0x0 in extern-address space (0x20))


<SimulationManager with 3 deadended>

In [16]:
sim_mgr.active

[]

In [17]:
sim_mgr.move(from_stash='deadended', to_stash='authenticated', filter_func=lambda s: b'Welcome' in s.posix.dumps(1))
sim_mgr

<SimulationManager with 1 deadended, 2 authenticated>

In [18]:
sim_mgr.active

[]

In [19]:
sim_mgr.authenticated

[<SimState @ 0x801058>, <SimState @ 0x801058>]

In [24]:
import monkeyhex

for s in sim_mgr.deadended + sim_mgr.authenticated:
    print(hex(s.addr))


0x700030
0x801058
0x801058


In [25]:
sim_mgr.one_deadended

<SimState @ 0x700030>

In [26]:
sim_mgr.one_authenticated

<SimState @ 0x801058>

In [27]:
sim_mgr.mp_authenticated

MP([<SimState @ 0x801058>, <SimState @ 0x801058>])

In [28]:
sim_mgr.mp_authenticated.posix.dumps(0)

MP([b'\x00\x00\x00\x00\x00\x00\x00\x00\x00SOSNEAKY\x00', b'\x00\x00\x00\x00\x00\x00\x00\x00\x00@@@@\x80@@@\x00'])

In [30]:
project = angr.Project("examples/CSCI-4968-MBE/challenges/crackme0x00a/crackme0x00a")

sim_mgr = project.factory.simgr()

In [31]:
sim_mgr.explore(find=lambda s: b"Congrats" in s.posix.dumps(1))


WARNING | 2022-03-13 10:03:10,859 | angr.storage.memory_mixins.default_filler_mixin | Filling register edi with 4 unconstrained bytes referenced from 0x8048571 (__libc_csu_init+0x1 in crackme0x00a (0x8048571))
WARNING | 2022-03-13 10:03:10,878 | angr.storage.memory_mixins.default_filler_mixin | Filling register ebx with 4 unconstrained bytes referenced from 0x8048573 (__libc_csu_init+0x3 in crackme0x00a (0x8048573))


<SimulationManager with 1 active, 1 found>

In [32]:
found_state = sim_mgr.found[0]

print(found_state.posix.dumps(1))

b'Enter password: Congrats!\n'


In [35]:
flag = found_state.posix.dumps(0)

In [36]:
print(flag)

b'g00dJ0B!\x00\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd8\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9\xd9'
